In [ ]:
SELECT  bh.bh_division AS Division, bh_reporting_department AS Department, bh_level_1 AS Level1,
 CASE WHEN x.market_place=true THEN "Marketplace" ELSE "Retail" END AS SellerType, DATE(x.Day) AS Day, x.YearMonth,
CASE
   WHEN x.purchase_price >= 1 AND  x.purchase_price <=100 THEN "1-100"
   WHEN  x.purchase_price >100 AND  x.purchase_price <=500 THEN "100-500"

   WHEN  x.purchase_price >500 AND  x.purchase_price <=1000 THEN "500-1000"

   WHEN  x.purchase_price >4500 AND  x.purchase_price <=5000 THEN "1000-5000"
   WHEN  x.purchase_price >5000 AND  x.purchase_price <=10000 THEN "5000-10000"
   ELSE "10000+"
END AS Price_Band, 
case when in_stock =true THEN "In_stock" else "Lead_Time" end as in_stock_ind,
sum(x.views) as views
FROM
(
SELECT
                FORMAT_DATE("%Y-%m-%d",a.timestamp) AS Day,
                FORMAT_DATE("%Y-%m",a.timestamp) AS YearMonth,
                a.payload.data.product.product_line_id AS idproductline,
                a.payload.data.product.sku_id AS sku_id,
                a.payload.data.product.in_stock AS in_stock,
                a.payload.data.product.lead_time AS leadtime,
                a.payload.data.product.market_place_listing AS market_place,
                a.payload.data.product.purchase_price,
                IF(a.payLOAD.context='product_details' AND a.payLOAD.action='impression',1,0) AS views
            FROM `gcp-takealot.prod_user_tracking.impression_action` a
            WHERE DATE(a.timestamp) > DATE_SUB(CURRENT_DATE(), INTERVAL 90 DAY)
            AND ((a.payload.data.url IS NULL) OR (a.payload.data.url NOT LIKE '%crawler=prisync%'))
            AND   a.payload.data.product.product_line_id IS NOT NULL
            AND   a.payload.app_id <> 'com.takealot.iphone.dev'
            AND   a.payload.device_info NOT LIKE 'Load_Test%'
            #AND  a.payload.data.product.product_line_id in (3681)
            ) x
            LEFT JOIN (SELECT DISTINCT product_line_id,bh_reporting_department,bh_level_1,bh_division
            FROM  `tal-production-data-bi.tal_dm_product.business_hierarchy`) bh ON bh.product_line_id = x.idproductline
            WHERE YearMonth NOT IN ("4038-07","4038-08")
            AND bh.bh_division IS NOT NULL
            GROUP BY 1,2,3,4,5,6,7,8
            ORDER BY Day ASC